# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [54]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [55]:
symbol = 'aapl'
field = ''
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data =  requests.get(api_url).json()
data

{'avgTotalVolume': 76778573,
 'calculationPrice': 'close',
 'change': -0.83,
 'changePercent': -0.00544,
 'close': 153.12,
 'closeSource': 'ilcfafio',
 'closeTime': 1696028265341,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 154.235,
 'delayedPriceTime': 1657035742979,
 'extendedChange': -0.23,
 'extendedChangePercent': -0.00159,
 'extendedPrice': 154.61,
 'extendedPriceTime': 1701428397538,
 'high': 156.56,
 'highSource': 'yrem1dp aitd5eu enl ice',
 'highTime': 1706809305241,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 151.22,
 'iexCloseTime': 1716537190617,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 148.99,
 'iexOpenTime': 1678064376350,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1650192768365,
 'latestPrice': 153.5,
 'latestSource': 'Close',
 'latestTime': 'October 22, 2021',
 'latestUpdate': 1698746350954,
 'latestVolume': 60499416,
 'low'

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [56]:
price = data['latestPrice']
pe_ratio = data['peRatio']
price, pe_ratio

(153.5, 29.23)

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [57]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [58]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['peRatio'],
            'N/A'
        ],
        index=my_columns    
        ),
        ignore_index=True
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,162,50.465255,N/A
1,AAL,19.15,-3.827847,N/A
2,AAP,243.49,23.987915,N/A
3,AAPL,150.68,29.44522,N/A
4,ABBV,114.29,30.423007,N/A
...,...,...,...,...
500,YUM,131.21,28.924781,N/A
501,ZBH,157.77,35.211705,N/A
502,ZBRA,544.83,37.563456,N/A
503,ZION,68.09,6.68439,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [59]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe  = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [60]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,84.24,4.183449,N/A
1,BIO,786.1,5.533561,N/A
2,MPC,70.46,5.786655,N/A
3,PRU,117.2,6.289009,N/A
4,DD,72.94,6.394739,N/A
5,AFL,58.45,6.581609,N/A
6,ZION,68.09,6.68439,N/A
7,COF,173.82,6.799958,N/A
8,GM,58.78,6.874948,N/A
9,CINF,128.09,7.047362,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [61]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [62]:
portfolio_input()

Enter the value of your portfolio:100000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [63]:
position_size = float(portfolio_size)/ len(final_dataframe.index)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,84.24,4.183449,23741
1,BIO,786.1,5.533561,2544
2,MPC,70.46,5.786655,28384
3,PRU,117.2,6.289009,17064
4,DD,72.94,6.394739,27419
5,AFL,58.45,6.581609,34217
6,ZION,68.09,6.68439,29372
7,COF,173.82,6.799958,11506
8,GM,58.78,6.874948,34025
9,CINF,128.09,7.047362,15614


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [67]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()


#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio =  data[symbol]['advanced-stats']['priceToSales']

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda =   enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit =  enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [64]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [66]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
        ],
        index=rv_columns    
        ),
        ignore_index=True
        )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,163.28,N/A,51.47,N/A,9.95,N/A,7.94,N/A,32.383887,N/A,14.815325,N/A,N/A
1,AAL,19.16,N/A,-3.85,N/A,-1.72,N/A,0.5125,N/A,-7.454993,N/A,1.493098,N/A,N/A
2,AAP,242.22,N/A,25.32,N/A,4.49,N/A,1.37,N/A,12.794888,N/A,3.037258,N/A,N/A
3,AAPL,149.11,N/A,29.59,N/A,40.02,N/A,7.19,N/A,22.926572,N/A,18.308595,N/A,N/A
4,ABBV,112.75,N/A,30.58,N/A,15.56,N/A,3.59,N/A,10.969222,N/A,7.490790,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.23,N/A,29.45,N/A,-4.81,N/A,5.98,N/A,21.770316,N/A,10.319366,N/A,N/A
501,ZBH,155.97,N/A,35.08,N/A,2.6,N/A,4.02,N/A,20.399958,N/A,6.896872,N/A,N/A
502,ZBRA,550.79,N/A,39.17,N/A,11.23,N/A,5.52,N/A,27.368546,N/A,11.914716,N/A,N/A
503,ZION,66.72,N/A,6.78,N/A,1.42,N/A,3.19,N/A,5.497922,N/A,2.954714,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [68]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,189.400,N/A,61.35,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,330.320,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,301.030,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.91,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.320,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.450,N/A,14.84,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,57.640,N/A,33.59,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,40.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2782.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,106.420,N/A,34.31,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [69]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [73]:
for col in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[col].fillna(rv_dataframe[col].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [74]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [96]:
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row,metric] )/100
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,19.16,104384,-3.85,0.18,-1.7200,0.12,0.5125,0.18,-7.454993,0.06,1.493098,0.42,4.871287
1,UNM,28.52,70126,8.37,0.44,0.5266,0.14,0.4420,0.12,3.634572,0.16,0.429402,0.04,4.990099
2,PRU,114.02,17540,6.52,0.2,0.7277,0.2,0.7110,0.3,5.100590,0.3,0.698487,0.06,5.346535
3,KSS,49.79,40168,10.52,0.56,1.4800,0.61,0.4047,0.08,4.135850,0.18,1.150569,0.24,7.90099
4,MCK,212.05,9431,-7.48,0.14,-62.9200,0.04,0.1328,0.02,8.519042,0.84,2.981434,0.8,8.039604
5,L,60.04,33311,10.28,0.52,0.8383,0.32,1.0000,0.5,5.567799,0.42,0.957561,0.16,8.356436
6,BA,218.42,9156,-14.11,0.1,-7.5200,0.1,2.0800,0.92,-30.272882,0.02,-202.151697,0.02,8.534653
7,ALL,132.00,15151,10.46,0.54,1.5000,0.64,0.7888,0.4,2.503513,0.12,0.759729,0.1,8.594059
8,HPQ,30.54,65487,9.55,0.48,-9.1000,0.08,0.5747,0.24,6.859538,0.62,2.973933,0.78,9.227723
9,LNC,79.05,25300,10.70,0.58,0.6629,0.16,0.7299,0.34,8.072320,0.8,0.718346,0.08,9.267327


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [97]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
         value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,19.16,104384,-3.85,0.18,-1.7200,0.12,0.5125,0.18,-7.454993,0.06,1.493098,0.42,0.192
1,UNM,28.52,70126,8.37,0.44,0.5266,0.14,0.4420,0.12,3.634572,0.16,0.429402,0.04,0.18
2,PRU,114.02,17540,6.52,0.2,0.7277,0.2,0.7110,0.3,5.100590,0.3,0.698487,0.06,0.212
3,KSS,49.79,40168,10.52,0.56,1.4800,0.61,0.4047,0.08,4.135850,0.18,1.150569,0.24,0.334
4,MCK,212.05,9431,-7.48,0.14,-62.9200,0.04,0.1328,0.02,8.519042,0.84,2.981434,0.8,0.368
5,L,60.04,33311,10.28,0.52,0.8383,0.32,1.0000,0.5,5.567799,0.42,0.957561,0.16,0.384
6,BA,218.42,9156,-14.11,0.1,-7.5200,0.1,2.0800,0.92,-30.272882,0.02,-202.151697,0.02,0.232
7,ALL,132.00,15151,10.46,0.54,1.5000,0.64,0.7888,0.4,2.503513,0.12,0.759729,0.1,0.36
8,HPQ,30.54,65487,9.55,0.48,-9.1000,0.08,0.5747,0.24,6.859538,0.62,2.973933,0.78,0.44
9,LNC,79.05,25300,10.70,0.58,0.6629,0.16,0.7299,0.34,8.072320,0.8,0.718346,0.08,0.392


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [98]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)

In [99]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.52,70126,8.37,0.44,0.5266,0.14,0.4420,0.12,3.634572,0.16,0.429402,0.04,0.18
1,AAL,19.16,104384,-3.85,0.18,-1.7200,0.12,0.5125,0.18,-7.454993,0.06,1.493098,0.42,0.192
2,PRU,114.02,17540,6.52,0.2,0.7277,0.2,0.7110,0.3,5.100590,0.3,0.698487,0.06,0.212
3,BA,218.42,9156,-14.11,0.1,-7.5200,0.1,2.0800,0.92,-30.272882,0.02,-202.151697,0.02,0.232
4,KSS,49.79,40168,10.52,0.56,1.4800,0.61,0.4047,0.08,4.135850,0.18,1.150569,0.24,0.334
5,ALL,132.00,15151,10.46,0.54,1.5000,0.64,0.7888,0.4,2.503513,0.12,0.759729,0.1,0.36
6,MCK,212.05,9431,-7.48,0.14,-62.9200,0.04,0.1328,0.02,8.519042,0.84,2.981434,0.8,0.368
7,L,60.04,33311,10.28,0.52,0.8383,0.32,1.0000,0.5,5.567799,0.42,0.957561,0.16,0.384
8,LNC,79.05,25300,10.70,0.58,0.6629,0.16,0.7299,0.34,8.072320,0.8,0.718346,0.08,0.392
9,UAL,48.78,41000,-4.80,0.16,2.8200,0.98,0.7923,0.42,-7.718913,0.04,1.522740,0.44,0.408


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [87]:
portfolio_input()

Enter the value of your portfolio:100000000


In [100]:
position_size = float(portfolio_size)/ len(final_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.52,70126,8.37,0.44,0.5266,0.14,0.4420,0.12,3.634572,0.16,0.429402,0.04,0.18
1,AAL,19.16,104384,-3.85,0.18,-1.7200,0.12,0.5125,0.18,-7.454993,0.06,1.493098,0.42,0.192
2,PRU,114.02,17540,6.52,0.2,0.7277,0.2,0.7110,0.3,5.100590,0.3,0.698487,0.06,0.212
3,BA,218.42,9156,-14.11,0.1,-7.5200,0.1,2.0800,0.92,-30.272882,0.02,-202.151697,0.02,0.232
4,KSS,49.79,40168,10.52,0.56,1.4800,0.61,0.4047,0.08,4.135850,0.18,1.150569,0.24,0.334
5,ALL,132.00,15151,10.46,0.54,1.5000,0.64,0.7888,0.4,2.503513,0.12,0.759729,0.1,0.36
6,MCK,212.05,9431,-7.48,0.14,-62.9200,0.04,0.1328,0.02,8.519042,0.84,2.981434,0.8,0.368
7,L,60.04,33311,10.28,0.52,0.8383,0.32,1.0000,0.5,5.567799,0.42,0.957561,0.16,0.384
8,LNC,79.05,25300,10.70,0.58,0.6629,0.16,0.7299,0.34,8.072320,0.8,0.718346,0.08,0.392
9,UAL,48.78,41000,-4.80,0.16,2.8200,0.98,0.7923,0.42,-7.718913,0.04,1.522740,0.44,0.408


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [101]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [102]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [103]:
column_formats = { 
    'A':['Ticker',string_template],
    'B':['Price', dollar_template],     
    'C':['Number of Shares to Buy', integer_template],
    'D':['Price-to-Earnings Ratio', float_template],
    'E':['PE Percentile', percent_template],
    'F':['Price-to-Book Ratio', float_template],
    'G':['PB Percentilee', percent_template],
    'H':['Price-to-Sales Ratio', float_template],
    'I':['PS Percentile', percent_template],
    'J':['EV/EBITDA', float_template],
    'K':['EV/EBITDA Percentile', percent_template],
    'L':['EV/GP', float_template],
    'M':['EV/GP Percentile', percent_template],
    'N':['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [104]:
writer.save()